In [3]:
user_queries = [
        "장기보험상품에서 암진단 후 지급기간은?",
        "장기보험상품에서 보장주기에 대해 알려줘",
        "장기보험상품에서 보장범위는 어떻게 되나요?", 
        "장기보험상품에서 보장 불가한 항목은 어떻게 되나요?",
        "장기보험상품에서 보장 가능한 병원등급은 어떻게 되나요?",
        "장기보험상품에서 보장하는 방식에서 몇 회까지 보장하는지 알려줘",
        "장기보험상품에서 보장하는 기준을 알려줘",
]

In [4]:
from openai import OpenAI

model = "qwen3:8b"
#일반보험_챗봇 id
# client = OpenAI(
#     api_key="ragflow-U5ZGEyNTdlNjkyODExZjBiODE2MDI0Mm", 
#     base_url=f"http://10.50.7.154:8080/api/v1/chats_openai/d0f38fb26c1811f0bbf00242ac150007"
# )
#장기보험_챗봇 id: 269d722c6c3a11f097260242ac150007
client = OpenAI(
    api_key="ragflow-U5ZGEyNTdlNjkyODExZjBiODE2MDI0Mm", 
    base_url=f"http://10.50.7.154:8080/api/v1/chats_openai/269d722c6c3a11f097260242ac150007"
)

def retrieve_and_generate(question, stream=False):
    completion = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "user", "content": question},
        ],
        stream=stream,
    )

    if stream:
        full_response = ""
        for chunk in completion:
            # Extract the content from the chunk
            if hasattr(chunk.choices[0].delta, "content") and chunk.choices[0].delta.content:
                content = chunk.choices[0].delta.content
                full_response += content
                # Replace \n and \n\n with actual line breaks and print
                print(content.replace("\\n", "\n"), end="")
        print("\n\nFinal Processed Response:")
        print(full_response.replace("\\n", "\n")) 
    else:
        content = completion.choices[0].message.content
        print(content)

In [ ]:
retrieve_and_generate(user_queries[0], stream=False)

In [11]:
from langfuse import Langfuse

langfuse = Langfuse(
  secret_key="sk-lf-8b2404f1-6a48-4dfb-b62e-5db0cc0f28c8",
  public_key="pk-lf-3cf05432-2a74-47f0-bf6c-fb5859c5cb50",
  host="http://10.50.3.31:3000"
)

In [7]:
from langfuse.openai import OpenAI
from langchain_core.output_parsers import StrOutputParser

#장기보험_챗봇 id: 269d722c6c3a11f097260242ac150007
client = OpenAI(
	api_key="ragflow-U5ZGEyNTdlNjkyODExZjBiODE2MDI0Mm", 
    base_url=f"http://10.50.7.154:8080/api/v1/chats_openai/269d722c6c3a11f097260242ac150007"
)

def retrieve_and_generate(question, stream=False):
    completion = client.chat.completions.create(
        model="qwen3:8b",
        messages=[
            {"role": "user", "content": question},
        ],
        stream=stream,
    )
    output_parser = StrOutputParser()

    if stream:
        full_response = ""
        for chunk in completion:
            # Extract the content from the chunk
            if hasattr(chunk.choices[0].delta, "content") and chunk.choices[0].delta.content:
                content = chunk.choices[0].delta.content
                output = output_parser.parse(content)
                # 파싱된 결과 출력
                print(output, end="")
                full_response += content
        #output = output_parser.parse(full_response)
        #print(output)
    else:
        content = completion.choices[0].message.content
        output = output_parser.parse(content)
        print(output)

In [ ]:
retrieve_and_generate(user_queries[1], stream=True)

In [ ]:
from ragflow_sdk import RAGFlow

api_key = "ragflow-U5ZGEyNTdlNjkyODExZjBiODE2MDI0Mm"
base_url = "http://10.50.7.154:8080"

rag_object = RAGFlow(api_key=api_key, base_url=base_url)

# 모든 지식베이스 조회
dataset_ids = []
for dataset in rag_object.list_datasets(name="장기보험상품"):
    print(dataset)
    dataset_ids.append(dataset.id)

chunks = rag_object.retrieve(dataset_ids=dataset_ids, question=user_queries[0])
print(f'len: {len(chunks)}')
for c in chunks:
    print(f'{c.document_name}-{len(c.content)}: {c.content}')
# datasets = rag_object.list_datasets(name="장기보험상품")
# dataset_id = datasets[0].id
# print(dataset_id)
# rag_object.retrieve(dataset_ids=[dataset_id], question=user_queries[0])

In [ ]:
from ragflow_sdk import RAGFlow

api_key = "ragflow-U5ZGEyNTdlNjkyODExZjBiODE2MDI0Mm"
base_url = "http://10.50.7.154"
rag_object = RAGFlow(api_key=api_key, base_url=f"{base_url}:9380")
assistant = rag_object.list_chats(name="장기보험_챗봇")
assistant = assistant[0]
session = assistant.create_session()    

print("\n==================== 장기보험 챗봇 =====================\n")
print("Hello. What can I do for you?")

while True:
    question = input("\n==================== User =====================\n> ")
    print("\n==================== 장기보험 챗봇 =====================\n")
    
    cont = ""
    for ans in session.ask(question, stream=True):
        print(ans.content[len(cont):], end='', flush=True)
        cont = ans.content